In [56]:
import numpy as np
import pandas as pd
from climateUtils import eca_read_station_like_file
import matplotlib.pyplot as plt

In [19]:
stations_pres = eca_read_station_like_file("eca_data/ECA_blend_station_pp.txt")
stations_prec = eca_read_station_like_file("eca_data/ECA_blend_station_rr.txt")
stations_tavg = eca_read_station_like_file("eca_data/ECA_blend_station_tg.txt")
stations_rads = eca_read_station_like_file("eca_data/ECA_blend_station_qq.txt")

stations_dict = {'pp' : stations_pres, 'rr' : stations_prec, 'tg' : stations_tavg, 'qq' : stations_rads}

In [39]:
dataframes = []
for measure, stations in stations_dict.items():
    for _,row in stations.iterrows():
        if(_ % 500 == 0):
            print(_)
        stat_id = row['STAID']
        filename = "eca_data/ECA_blend_%s/%s_STAID%06i.txt" % (measure, measure.upper(),stat_id)
        df = eca_read_station_like_file(filename)
        df['DATE'] = pd.to_datetime(df['DATE'], format="%Y%m%d")
        df[df == -9999] = np.nan
        df.dropna(axis=0, inplace=True)
        df = df[['STAID', 'DATE', measure.upper()]]
        df.rename({'STAID': 'station_id', 'DATE' : 'date', measure.upper() : 'value' }, axis=1, inplace=True)
        df['measure'] = measure
        df.set_index(['station_id', 'measure', 'date'], drop=True, inplace=True)
        dataframes.append(df)

0
500
1000
1500
2000
2500
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
0
500
1000
1500
2000


In [40]:
dataframes = pd.concat(dataframes, axis=0)

In [45]:
measure_x = 'pp'
measure_y = 'tg'
subset = pd.merge(dataframes.xs(measure_x, level='measure'), dataframes.xs(measure_y, level='measure'),left_index=True, right_index=True, how='inner')

In [49]:
subset

value_x  value_y
station_id date                        
1          1961-01-01  10118.0     -5.0
           1961-01-02  10081.0      7.0
           1961-01-03   9935.0      7.0
           1961-01-04   9975.0      6.0
           1961-01-05  10019.0      3.0
...                        ...      ...
26662      2024-03-27   9973.0    134.0
           2024-03-28  10015.0    133.0
           2024-03-29  10124.0    135.0
           2024-03-30  10095.0    191.0
           2024-03-31  10074.0    194.0

[34066394 rows x 2 columns]

In [59]:
for station_id, group in subset.groupby('station_id'):
    group_indexed = group.reset_index(level='station_id', drop=True)
    rolling_cov = group_indexed['value_x'].rolling('30D').cov(group_indexed['value_y'])
    rolling_varx = group_indexed['value_x'].rolling('30D').var()
    rolling_vary = group_indexed['value_y'].rolling('30D').var()
    break